In [ ]:
# Initial Thoughts


# have following variables

# Age - continuous
# Acid Level - continuous
# x ray res - binary
# tumour size - binary
# tumour grade - binary

# Predict nodal involvement - binary = logistic regression


# Do boxplots of age, acid level vs nodal involvement, 
# Observe interaction plots and some more?